In [2]:
API_key = '18b2f98f889d42b8b08cc02fafa8806b'

In [10]:
import concurrent.futures
import requests
import time
import json
from datetime import datetime
import logging
import os
import sys
import pandas as pd

class FastAPI:
    def __init__(self, api_key, config_file='config_selenium.json'):
        self.api_key = api_key
        self.base_url = "https://api.rawg.io/api/games"

        # Загружаем конфигурацию логирования
        self._setup_logging_from_config(config_file)
        self.logger = logging.getLogger(__name__)

    def _setup_logging_from_config(self, config_file):
        try:
            with open(config_file, 'r', encoding='utf-8') as f:
                config = json.load(f)

            log_config = config.get('logging', {})

            # Проверяем включено ли логирование
            if not log_config.get('enabled', True):
                logging.getLogger().disabled = True
                return

            # Настройка уровня и обработчиков
            level = getattr(logging, log_config.get('level', 'INFO'))

            handlers = []
            formatter = logging.Formatter(
                '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
            )

            if log_config.get('log_to_file', True):
                file_handler = logging.FileHandler(
                    log_config.get('filename', 'api_client.log'),
                    encoding='utf-8'
                )
                file_handler.setFormatter(formatter)
                handlers.append(file_handler)

            if log_config.get('log_to_console', True):
                console_handler = logging.StreamHandler(sys.stdout)
                console_handler.setFormatter(formatter)
                handlers.append(console_handler)

            logging.basicConfig(level=level, handlers=handlers)

        except Exception as e:
            logging.basicConfig(
                level=logging.INFO,
                format='%(asctime)s - %(levelname)s - %(message)s'
            )

    def get_years_parallel(self, start_year=2018, end_year=2019):
        years = list(range(start_year, end_year + 1))
        all_games = []

        self.logger.info(f"Начинаем загрузку данных за {start_year}-{end_year} годы")
        self.logger.info(f"Всего лет для загрузки: {len(years)}")

        with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
            future_to_year = {
                executor.submit(self._get_games_for_year, year):
                year for year in years
            }

            completed = 0
            total = len(years)

            for future in concurrent.futures.as_completed(future_to_year):
                year = future_to_year[future]
                try:
                    year_games = future.result()
                    all_games.extend(year_games)
                    completed += 1
                    self.logger.info(f"Успешно для {year} год: {len(year_games)} игр | Прогресс: {completed}/{total}")
                except Exception as e:
                    completed += 1
                    self.logger.error(f"Ошибка для {year} года: {e} | Прогресс: {completed}/{total}")

        return all_games

    def _get_games_for_year(self, year):
        games = []
        page = 1
        max_pages = 25

        while page <= max_pages:
            params = {
                'key': self.api_key,
                'dates': f"{year}-01-01,{year}-12-31",
                'page': page,
                'page_size': 40
            }

            try:
                response = requests.get(self.base_url, params=params, timeout=30)

                if response.status_code == 502:
                    self.logger.warning(f"Ошибка 502 для {year} года, страница {page}, ждем")
                    time.sleep(3)
                    continue

                response.raise_for_status()
                data = response.json()

                page_games = data.get('results', [])
                if not page_games:
                    break

                games.extend(page_games)

                if not data.get('next'):
                    break

                page += 1
                time.sleep(0.8)

            except requests.exceptions.RequestException as e:
                self.logger.error(f"Ошибка для {year} года, страница {page}: {e}")
                break
            except Exception as e:
                self.logger.error(f"Ошибка для {year} года: {e}")
                break

        return games

def run_in_jupyter(start_year=2017, end_year=2019):
    api_key = "18b2f98f889d42b8b08cc02fafa8806b"
    client = FastAPI(api_key)

    client.logger.info("Запуск загрузки данных")
    start_time = time.time()

    # Вызов основного метода
    all_games = client.get_years_parallel(start_year, end_year)

    end_time = time.time()
    total_time = end_time - start_time

    client.logger.info(f"Загрузка завершена!")
    client.logger.info(f"Собрано {len(all_games)} игр")
    client.logger.info(f"Затрачено времени: {total_time:.1f} секунд")
    client.logger.info(f"Скорость: {len(all_games)/total_time:.1f} игр/секунду")

    filename = f'games_parallel_{start_year}_{end_year}.csv'

    df = pd.DataFrame(all_games)
    df.to_csv(filename, index=False, encoding='utf-8-sig')

    client.logger.info(f"Данные сохранены в файл: {filename}")

    return all_games


In [12]:
for year in range(2025, 2026):
    run_in_jupyter(start_year=year, end_year=year)

2025-10-27 00:26:16,228 - __main__ - INFO - Запуск загрузки данных
2025-10-27 00:26:16,231 - __main__ - INFO - Начинаем загрузку данных за 2025-2025 годы
2025-10-27 00:26:16,232 - __main__ - INFO - Всего лет для загрузки: 1
2025-10-27 00:27:14,041 - __main__ - INFO - Успешно для 2025 год: 1000 игр | Прогресс: 1/1
2025-10-27 00:27:14,045 - __main__ - INFO - Загрузка завершена!
2025-10-27 00:27:14,048 - __main__ - INFO - Собрано 1000 игр
2025-10-27 00:27:14,051 - __main__ - INFO - Затрачено времени: 57.8 секунд
2025-10-27 00:27:14,055 - __main__ - INFO - Скорость: 17.3 игр/секунду
2025-10-27 00:27:14,721 - __main__ - INFO - Данные сохранены в файл: games_parallel_2025_2025.csv
